# Ejercicio de Procesado de Lenguaje Natural
En este ejercicio vamos a utilizar las técnicas aprendidas de NLP para clasificar la tipología de un conjunto de reclamaciones ciudadanas.  
Utilizaremos un modelo TF-IDF básico, y luego probaremos con bigrams y una reducción de dimensionalidad LSA para ver si obtenemos alguna mejora.

### Importación librerías

In [103]:
import pandas as pd
import numpy as np
import spacy
pd.set_option('display.max_colwidth', -1)
#Importa el resto de librerías necesarias
import re
import string
import matplotlib.pyplot as plt
import gensim
nlp = spacy.load('es_core_news_md')
%matplotlib inline

### Carga de los datos
El archivo CSV de datos tiene tres columnas:  
- Observaciones: el texto (incidencia) a clasificar
- Tipología: la clase (etiqueta) de cada incidencia
- Original: característica binaria que no se usa en este problema

In [104]:
datos=pd.read_csv("./incidencias.csv", sep=";")
datos.head()

,Tipología,Observaciones,Original
0,Vía Pública,hay coches mal aparcados en la calle de los mayos,N
1,Vía Pública,las calles están llenas de mierdas de los perros por todas partes,N
2,Vía Pública,hay un socavón en la avenida de ademuz,N
3,Vía Pública,Rejilla metálica al lado de la carretera frente a casa de Catalina y Pepe en mal estado. Se ha producido una caída.,S
4,Vía Pública,Cada vez que hay una tormenta se llena la calle de tierra y piedras. Considero que deberían buscar una rápida solución. Gracias,S


Muestra información del DataFrame y la cuenta de muestras en cada clase. ¿cuántas clases distintas hay? ¿están balanceadas?

In [105]:
#completarç
datos['Tipología'].value_counts()

Mobiliario Urbano                31
Alumbrado                        31
Parques y jardines               29
Vía Pública                      28
Limpieza                         28
Agua                             27
Alcantarillado                   26
Plagas de insectos y roedores    20
Name: Tipología, dtype: int64

### Limpieza del texto
Programa una función para limpiar el texto en los siguientes términos:  
- Elimina los números (expresión regular `r'\d+'`)  
- Elimina los signos de puntuación
- Convierte todas las palabras a minúscula

In [106]:
#completar
import re, string, spacy
nlp=spacy.load('es_core_news_md')

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text):
    text = re.sub(r'\d+', '', text) #elimina los números
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space and len(tok)>2]
    filtered_tokens = [pattern2.sub('', token) for token in tokens]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

def lemmatize_text(text):
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    
    return lemmatized_text

#Elimina los números (expresión regular r'\d+')
#Elimina los signos de puntuación
#Convierte todas las palabras a minúscula

Aplica la función de limpieza a la columna 'Observaciones' del DataFrame

In [107]:
#completar
datos.Observaciones=datos.Observaciones.apply(clean_text)

### Funciones auxiliares
Completa estas funciones para calcular la matriz BoW y TF-IDF del Corpus de texto. La función `train_predict_evaluate_model` ya está definida por ti.

In [108]:
#función para extraer el modelo TF-IDF del corpus

def bow_extractor(corpus, ngram_range=(1,1), min_df=1, max_df=1.0):
    """Función que ajusta un modelo BoW sobre un corpus de texto
    y devuelve el modelo y la matriz BoW
    El corpus se debe pasar como una lista de strings."""
    #COMPLETAR
    vectorizer = CountVectorizer()
    features = vectorizer.fit_transform(corpus)
    
    return vectorizer, features
    
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_extractor(corpus, ngram_range=(1,1), min_df=1, max_df=1.0):
    """Función que ajusta un modelo TF-IDF sobre un corpus de texto
    y devuelve el modelo y la matriz TF-IDF
    El corpus se debe pasar como una lista de strings."""   
    #COMPLETAR
    vectorizer = TfidfVectorizer()
    features = vectorizer.fit_transform(corpus)
    
    return vectorizer, features

    
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    '''Función que entrena un clasificador, lo evalúa sobre un conjunto
    de test, y muestra su rendimiento'''
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    print(classification_report(test_labels, predictions))
    return predictions 

### División del conjunto de datos
Divide los datos en un conjunto de entrenamiento (`X_train`, `y_train`) y test (`X_test`, `y_test`) en una proporción 70-30:

In [109]:
#Completar
from sklearn.model_selection import train_test_split

# Split data into training and test sets
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test = train_test_split(datos['Observaciones'], 
                                                    datos['Tipología'],
                                                    test_size=0.3,
                                                    random_state=0)

## Clasificación
Entrena un clasificador sobre el conjunto de TRAIN y valida en TEST.  
Prueba con las matrices BoW y TF-IDF, y utiliza los siguientes clasificadores de la librería `scikit-learn`:  
```python
modelLR = LogisticRegression()
modelNB = GaussianNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=100)
```
Guarda las predicciones de todos los modelos (luego la usarás para mostrar la matriz de confusión)

In [110]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

## TF_IDF

In [111]:
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(X_train)  
tfidf_test_features = tfidf_vectorizer.transform(X_test)

In [112]:
tfidf_train_features=tfidf_train_features.toarray()
tfidf_test_features=tfidf_test_features.toarray()

In [113]:
modelLR = LogisticRegression()
#modelNB = MultinomialNB()
modelNB = GaussianNB()
modelSVM = SGDClassifier(loss='hinge', max_iter=100)

#Entrenamos los 3 clasificadores con las características TF-IDF
modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('Linear SVM', modelSVM)]
for m, clf in modelos:
    print('Modelo {} con características TF-IDF'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=tfidf_train_features,
                                           train_labels=y_train,
                                           test_features=tfidf_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características TF-IDF
                               precision    recall  f1-score   support

                         Agua       0.80      0.50      0.62         8
               Alcantarillado       1.00      0.50      0.67        10
                    Alumbrado       0.69      0.82      0.75        11
                     Limpieza       1.00      0.21      0.35        14
            Mobiliario Urbano       0.32      0.86      0.46         7
           Parques y jardines       0.33      0.80      0.47         5
Plagas de insectos y roedores       0.75      1.00      0.86         3
                  Vía Pública       0.40      0.25      0.31         8

                     accuracy                           0.55        66
                    macro avg       0.66      0.62      0.56        66
                 weighted avg       0.72      0.55      0.54        66

Modelo Naive Bayes con características TF-IDF
                               precision    r

## BOW

In [114]:
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer, bow_train_features = bow_extractor(X_train)
bow_test_features = bow_vectorizer.transform(X_test)

In [115]:
bow_train_features=bow_train_features.toarray()
bow_test_features=bow_test_features.toarray()

In [116]:
for m, clf in modelos:
    print('Modelo {} con características BoW'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=bow_train_features,
                                           train_labels=y_train,
                                           test_features=bow_test_features,
                                           test_labels=y_test)

Modelo Logistic Regression con características BoW
                               precision    recall  f1-score   support

                         Agua       0.86      0.75      0.80         8
               Alcantarillado       0.57      0.40      0.47        10
                    Alumbrado       0.90      0.82      0.86        11
                     Limpieza       1.00      0.29      0.44        14
            Mobiliario Urbano       0.32      0.86      0.46         7
           Parques y jardines       0.33      0.80      0.47         5
Plagas de insectos y roedores       0.75      1.00      0.86         3
                  Vía Pública       0.67      0.25      0.36         8

                     accuracy                           0.58        66
                    macro avg       0.67      0.65      0.59        66
                 weighted avg       0.73      0.58      0.57        66

Modelo Naive Bayes con características BoW
                               precision    recall 

¿Cuál es el modelo que mejor funciona?  
Muestra la matriz de confusión sobre el conjunto de test usando el siguiente código (debes sustituir `prediccion` por el array de predicciones de tu mejor modelo):

### Modelo Linear SVM con características TF-IDF  el mejor rendimiento

In [117]:
prediccion = modelSVM.predict(tfidf_test_features)


In [118]:
#Matriz de confusión
resultados = pd.DataFrame({'Real': y_test, 'Prediccion': prediccion})
pd.crosstab(resultados['Real'], resultados['Prediccion'],margins=True)

Prediccion,Agua,Alcantarillado,Alumbrado,Limpieza,Mobiliario Urbano,Parques y jardines,Plagas de insectos y roedores,Vía Pública,All
Real,,,,,,,,,
Agua,5,0,0,2,0,1,0,0,8
Alcantarillado,1,5,0,2,1,1,0,0,10
Alumbrado,0,0,5,5,1,0,0,0,11
Limpieza,0,0,0,11,2,0,0,1,14
Mobiliario Urbano,0,0,0,4,2,1,0,0,7
Parques y jardines,0,0,0,2,1,2,0,0,5
Plagas de insectos y roedores,0,0,0,1,0,0,2,0,3
Vía Pública,0,0,0,6,1,0,0,1,8
All,6,5,5,33,8,5,2,2,66


### Mejoras del clasificador
Como actividad opcional, intenta mejorar los resultados de estos clasificadores utilizando otras técnicas, como por ejemplo:  
- Distintos clasificadores a los utilizados
- Realizando una reducción de dimensionalidad LDA o LSA del texto
- Utilizando una matriz de características basada en Word Vectors o Paragraph Vectors

### 1- Intento mejorar con unigrams y bigrams, pero empeora la predicción

In [119]:
#COMPLETAR

vect_ngrams = CountVectorizer(min_df=2, ngram_range=(1,2)).fit(X_train)

X_train_vectorized = vect_ngrams.transform(X_train)
X_test_vectorized = vect_ngrams.transform(X_test)

len(vect_ngrams.get_feature_names())

310

In [120]:
feature_names = np.array(vect_ngrams.get_feature_names())
np.random.seed(1234)
np.random.choice(feature_names, 10)

array(['water', 'pasa', 'vegetación', 'carretera frente', 'parque',
       'lavadero', 'lado', 'menos', 'todos los', 'problema'], dtype='<U18')

In [121]:
#from sklearn.linear_model import LogisticRegression
modelSVM = SGDClassifier(loss='hinge', max_iter=100)
#modelLR = LogisticRegression(solver='liblinear')
#Entrenamos el modelo con el conjunto de train
modelSVM.fit(X_train_vectorized, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=100, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [122]:
prediccion = modelSVM.predict(X_test_vectorized)

In [123]:
from sklearn.metrics import classification_report

print(classification_report(y_test, prediccion))

                               precision    recall  f1-score   support

                         Agua       0.78      0.88      0.82         8
               Alcantarillado       0.86      0.60      0.71        10
                    Alumbrado       0.88      0.64      0.74        11
                     Limpieza       0.71      0.71      0.71        14
            Mobiliario Urbano       0.60      0.86      0.71         7
           Parques y jardines       0.12      0.20      0.15         5
Plagas de insectos y roedores       0.40      0.67      0.50         3
                  Vía Pública       0.40      0.25      0.31         8

                     accuracy                           0.62        66
                    macro avg       0.59      0.60      0.58        66
                 weighted avg       0.66      0.62      0.63        66



### 2- Voy a probar con otro modelo de Naive Bayes, tampoco mejora

In [131]:
# BOW
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_extractor(corpus, ngram_range=(1,1), min_df=1, max_df=1.0):
    """Función que ajusta un modelo TF-IDF sobre un corpus de texto
    y devuelve el modelo y la matriz TF-IDF
    El corpus se debe pasar como una lista de strings."""   
    #COMPLETAR
    vectorizer = TfidfVectorizer()
    features = vectorizer.fit_transform(corpus)
    
    return vectorizer, features

tfidf_vectorizer, tfidf_train_features = tfidf_extractor(X_train)  
tfidf_test_features = tfidf_vectorizer.transform(X_test)

In [133]:
from sklearn.naive_bayes import MultinomialNB
modelNBM = MultinomialNB()

modelNBM.fit(tfidf_train_features, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [134]:
prediccion = modelNBM.predict(tfidf_test_features)

In [135]:
print(classification_report(y_test, prediccion))

                               precision    recall  f1-score   support

                         Agua       0.80      0.50      0.62         8
               Alcantarillado       1.00      0.40      0.57        10
                    Alumbrado       0.64      0.82      0.72        11
                     Limpieza       1.00      0.07      0.13        14
            Mobiliario Urbano       0.29      0.86      0.43         7
           Parques y jardines       0.24      0.80      0.36         5
Plagas de insectos y roedores       1.00      0.67      0.80         3
                  Vía Pública       1.00      0.25      0.40         8

                     accuracy                           0.48        66
                    macro avg       0.75      0.55      0.50        66
                 weighted avg       0.78      0.48      0.47        66



### 3- He probado con Word embeding, pero tampoco mejora el modelo original

In [141]:
tokens=nlp(datos.Observaciones[1])
tokens[1].vector.shape

(50,)

In [142]:
parraf=nlp(datos.Observaciones[1])
parraf.vector.shape

(50,)

In [143]:
#Cada vector tiene un tamaño de 50, por tanto hay que crear una matriz de
#tamaño (nº documentos,50) para guardar el promedio de los vectores de cada tweet
#y guardar en cada fila el correspondiente vector promedio
word_embeddings=np.zeros((len(datos.Observaciones), parraf.vector.shape[0]))

In [144]:
#Spacy ya calcula el promedio de los vectores de un documento en Doc.vector
vectors = [nlp(parraf).vector for parraf in datos.Observaciones]
for i,vector in enumerate(vectors):
    word_embeddings[i,:]=vector

In [145]:
word_embeddings.shape

(220, 50)

In [146]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(word_embeddings, 
                                                    datos['Tipología'], 
                                                    test_size=0.3,
                                                    random_state=0)

In [147]:
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    '''Función que entrena un clasificador, lo evalúa sobre un conjunto
    de test, y muestra su rendimiento'''
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    print(classification_report(test_labels, predictions))
    return predictions 


In [148]:
#entrenamos clasificadores con modelos word embeddings
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

modelLR = LogisticRegression()
#modelNB = MultinomialNB()
modelNB = GaussianNB()
modelSVM = SVC(gamma='scale', C=2)

modelos = [('Logistic Regression', modelLR),
           ('Naive Bayes', modelNB),
           ('RFB SVM', modelSVM)]



for m, clf in modelos:
    print('Modelo {} con características Word Embeddings promediados'.format(m))
    tfidf_predictions = train_predict_evaluate_model(classifier=clf,
                                           train_features=X_train,
                                           train_labels=y_train,
                                           test_features=X_test,
                                           test_labels=y_test)


Modelo Logistic Regression con características Word Embeddings promediados
                               precision    recall  f1-score   support

                         Agua       0.83      0.62      0.71         8
               Alcantarillado       0.71      0.50      0.59        10
                    Alumbrado       0.46      0.55      0.50        11
                     Limpieza       0.78      0.50      0.61        14
            Mobiliario Urbano       0.25      0.29      0.27         7
           Parques y jardines       0.14      0.20      0.17         5
Plagas de insectos y roedores       0.38      1.00      0.55         3
                  Vía Pública       0.38      0.38      0.38         8

                     accuracy                           0.48        66
                    macro avg       0.49      0.50      0.47        66
                 weighted avg       0.55      0.48      0.50        66

Modelo Naive Bayes con características Word Embeddings promediados
   

C:\Users\anade\Anaconda3\envs\env_prueba\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
